In [1]:
from os import environ as env

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import dayofmonth
from pyspark.sql.types import *

In [2]:
env["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/iobruno/Vault/credentials/iobruno-training-gcp_terraform-admin.json"

## Spark Setup

In [3]:
spark = SparkSession.builder\
            .master("local[*]")\
            .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
            .config("spark.driver.memory", "2g")\
            .config("spark.executor.memory", "8g")\
            .config("spark.cores.max", 8) \
            .appName("pyspark-playground")\
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 00:07:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sparkContext.setLogLevel("ERROR")

In [5]:
spark._jsc\
    .hadoopConfiguration()\
    .set("google.cloud.auth.service.account.json.keyfile", env["GOOGLE_APPLICATION_CREDENTIALS"])

## Load Datasets from GCS

### FHV HV Dataset

In [6]:
fhvhv_schema = StructType([
    StructField("dispatching_base_num", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("SR_Flag", IntegerType(), True),
    StructField("Affiliated_base_number", StringType(), True)
])

In [7]:
fhvhv: DataFrame = spark.read\
                    .option("header", True)\
                    .schema(fhvhv_schema)\
                    .csv("gs://iobruno_datalake_raw/dtc_ny_taxi_tripdata/fhvhv/fhvhv_tripdata_2021-06.csv.gz")

In [8]:
fhvhv.createOrReplaceTempView('fhvhv')

### Zone Lookup Dataset

In [9]:
zones_schema = StructType([
    StructField("LocationID", IntegerType(), True),
    StructField("Borough", StringType(), True),
    StructField("Zone", StringType(), True),
    StructField("service_zone", StringType(), True)
])

In [10]:
zones: DataFrame = spark.read\
                    .option("header", True)\
                    .schema(zones_schema)\
                    .csv("gs://iobruno_datalake_raw/dtc_ny_taxi_tripdata/zone_lookup/taxi_zone_lookup.csv")

In [11]:
zones.createOrReplaceTempView('zones')

## Homework

In [12]:
spark.sql("""

    SELECT 
        dispatching_base_num,
        Affiliated_base_number as affiliated_base_num,
        PULocationID as pickup_location_id,
        DOLocationID as dropoff_location_id,
        SR_Flag as sr_flag,
        pickup_datetime,
        dropoff_datetime
    FROM 
        fhvhv
    
""").createOrReplaceTempView('fhvhv')

In [13]:
spark.sql("""
    
    SELECT
        LocationID as location_id,
        Borough as borough,
        Zone as zone,
        service_zone                
    FROM 
        zones
    
""").createOrReplaceTempView('zones')

### Question 1

**Install Spark and PySpark** 

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

In [14]:
spark.version

'3.3.2'

### Question 2

**HVFHW June 2021**

Read it with Spark using the same schema as we did in the lessons. We will use this dataset for all the remaining questions.  
Repartition it to 12 partitions and save it to parquet. What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB). Select the answer which most closely matches.  

In [15]:
df = spark.sql("""SELECT * FROM fhvhv""")

In [16]:
df.repartition(12)\
    .write\
    .option("compression", "snappy")\
    .mode("overwrite")\
    .parquet("/tmp/dtc/fhvhv-week5")

In [17]:
!ls -lh /tmp/dtc/fhvhv-week5/

total 589776
-rw-r--r--@ 1 iobruno  wheel     0B Mar  5 00:08 _SUCCESS
-rw-r--r--@ 1 iobruno  wheel    24M Mar  5 00:08 part-00000-f51983be-cec6-40ed-bc3f-198e2cfa39ac-c000.snappy.parquet
-rw-r--r--@ 1 iobruno  wheel    24M Mar  5 00:08 part-00001-f51983be-cec6-40ed-bc3f-198e2cfa39ac-c000.snappy.parquet
-rw-r--r--@ 1 iobruno  wheel    24M Mar  5 00:08 part-00002-f51983be-cec6-40ed-bc3f-198e2cfa39ac-c000.snappy.parquet
-rw-r--r--@ 1 iobruno  wheel    24M Mar  5 00:08 part-00003-f51983be-cec6-40ed-bc3f-198e2cfa39ac-c000.snappy.parquet
-rw-r--r--@ 1 iobruno  wheel    24M Mar  5 00:08 part-00004-f51983be-cec6-40ed-bc3f-198e2cfa39ac-c000.snappy.parquet
-rw-r--r--@ 1 iobruno  wheel    24M Mar  5 00:08 part-00005-f51983be-cec6-40ed-bc3f-198e2cfa39ac-c000.snappy.parquet
-rw-r--r--@ 1 iobruno  wheel    24M Mar  5 00:08 part-00006-f51983be-cec6-40ed-bc3f-198e2cfa39ac-c000.snappy.parquet
-rw-r--r--@ 1 iobruno  wheel    24M Mar  5 00:08 part-00007-f51983be-cec6-40ed-bc3f-198e2cfa39ac-c000.snappy.p

### Question 3

**Count records**  

How many taxi trips were there on June 15? Consider only trips that started on June 15.

In [18]:
spark.sql("""

    SELECT
        dayofmonth(pickup_datetime) as day_of_month,
        count(1) as num_trips
    FROM
        fhvhv
    WHERE
        dayofmonth(pickup_datetime) = 15
    GROUP BY
        dayofmonth(pickup_datetime)
    
""").take(1)

[Row(day_of_month=15, num_trips=452470)]

### Question 4: 

**Longest trip for each day**  

Now calculate the duration for each trip. How long was the longest trip in Hours?

In [19]:
spark.sql("""

    WITH tripdata AS (
        SELECT
            pickup_location_id,
            dropoff_location_id,
            pickup_datetime,
            dropoff_datetime,
            (CAST(dropoff_datetime as LONG) - CAST(pickup_datetime as LONG)) as duration_in_secs
        FROM
            fhvhv
    ),
    
    trip_duration AS (
        SELECT 
            (duration_in_secs/3600) as duration_in_hours,
            dense_rank() OVER( ORDER BY duration_in_secs DESC ) as rnk
        FROM 
            tripdata t
    )
    
    SELECT 
        td.duration_in_hours
    FROM 
        trip_duration td
    WHERE
        td.rnk = 1

""").take(1)

[Row(duration_in_hours=66.8788888888889)]

### Question 6: 

**Most frequent pickup location zone**

Load the zone lookup data into a temp view in Spark [Zone Data](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv).  
Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?

In [20]:
spark.sql("""
    
    WITH trip_count_per_location AS (
        SELECT 
            f.pickup_location_id,
            count(1) as num_trips,
            dense_rank() over (order by count(1) desc) as rnk
        FROM 
            fhvhv f
        GROUP BY 
            f.pickup_location_id
    ) 
    
    SELECT 
        t.pickup_location_id,
        pu.zone,
        t.num_trips,
        t.rnk
    FROM 
        trip_count_per_location t
    INNER JOIN 
        zones pu ON t.pickup_location_id = pu.location_id
    WHERE 
        t.rnk = 1
    
""").take(1)

[Row(pickup_location_id=61, zone='Crown Heights North', num_trips=231279, rnk=1)]